### Purpose

The low liquidity trading team wants a better idea of the volatility capture associated with the the market making agreements they have which involve an option component. In order to do this we need to compare the profitability achieved for a given SLA agreement with an option component against the theoretical value of the option. The puropse of this notebook is to provide a framework to do this comparison and provide a high level metric to show how successful LLT has been at extracting the theoretical value of the option. This metric is refered to as volatility capture.


Inputs:<br />
exchange_symbol - A dictionary of the following format {exchange: {base: [quote_1, quote_2]}}. Where the base quote combinations are the symbols we are interested in analysing and exchange is the exchange these symbols trade on.<br />
minutes - An integer representing the minutes window over which resampling occurs for realized volatility computation, in this notebook set to 15.<br />
interest_rate - The annualized continuous interest rate used for pricing in the Black Scholes (BS) call pricing.<br />
tardis_instruments - A table in clickhouse which contains a mapping from tardis instrument ids to instrument ids this table must contain the instruemnt being examined in the notebook.<br />
llt_option_terms - A table in clickhouse which contains all option terms both past and present. For the symbol the user is interested in there must be an option in this table.<br />
llt_option_withdrawal - A table in clickhouse which contains all withdrawals/deposits from the relevant llt options.
mv_tardis_publictrades_1m - A materialized view in clickhouse which contains tardis trade data resampled to a 1 minute time frame. For the symbols and options you are interested in analyzing this data must be avalible.<br />

Outputs:<br />
ot_interested - The preprosessed options terms dataframe with volatility capture.
ot_agg - The preprosessed options aggregated by option agreement with volatility capture.

### Methodology: 

Our measure of volatility capture is given as follows:

\begin{equation}
\pi^{cap} = \frac{\pi^{real}}{\pi^{theo}}
\end{equation}

Where:<br />
$\pi^{real}$ - Is the realized profit.<br />
$\pi^{theo}$ - Is our estimate of the theoretical profit<br />

Our estimate of $\pi^{cap}$ is then simply given by the BS call pricing equation:

\begin{aligned}
C(S_{t},K,t)=S_{t}N(d_{1})-Ke^{-r(T-t)}N(d_{2})\\
d_{1}=\frac{\ln \frac{S_{t}}{K} + (r + \frac{\sigma^2}{2})(T-t)}{\sigma\sqrt{T-t}}\\
d_{2}=d_{1}-\sigma\sqrt{T-t}
\end{aligned}

Where:<br />
$C(S_{t},K,t)$ - The call option value.<br />
$S_{t}$ - Is the digital assets price at time t.<br />
$N$ - Is the normal cdf.<br />
$K$ - Is the strike price of the option.<br />
$r$ - Is the risk free rate.<br />
$\sigma$ - Is the return volatility of the digital asset.<br />
$T$ - Is the expiration date of the option.<br />
$t$ - The current time.<br />

In practicality due to the structure of the option agreements $S_{t} = K$. As such the BS call equation given above reduces to:

\begin{aligned}
C(S_{t},K,t)=KN(d_{1})-Ke^{-r(T-t)}N(d_{2})\\
d_{1}=\frac{(r + \frac{\sigma^2}{2})(T-t)}{\sigma\sqrt{T-t}}\\
d_{2}=d_{1}-\sigma\sqrt{T-t}
\end{aligned}

Given the backwards looking nature of this study we also set $\sigma = \sigma_{real}$ where $\sigma_{real}$ is the realized volatility of digital asset over the lifetime of the option. note also that $t=0$ as wea re considering the value of the option at the inception date. We then arrive at our final equation for $\pi^{theo}$:

\begin{aligned}
\pi^{theo}=KN(d_{1})-Ke^{-rT}N(d_{2})\\
d_{1}=\frac{(r + \frac{{\sigma_{real}}^2}{2})T}{\sigma_{real}\sqrt{T}}\\
d_{2}=d_{1}-\sigma_{real}\sqrt{T}
\end{aligned}

In practicality the realized profitability is infered from the option withdrawals and deposits in the "P&L - Volume - Report" google sheet. The relevant data has been extracted manually and placed in the clickhouse database llt_option_withdrawal. Transformation of this data into a usable format then occurs within the LLTVolatilityCapture class. This includes: In the case of there being multiple markets for a given option splitting the PnL and loan amount associated with these markets linearly between these markets. In the case of their being multiple options over a given time window for a given client splitting the PnL and loan amount associated with this client linearly between these options. In the case that the option quote currency is not a usd equivalent converting this to usd by using the ratio of strike prices (this is likely a poor approach to conversion).<br />

When computing the realized volatility we use the saem methodology used in the LLT_Realized_Volatility.ipynb (found in the same directory as this notebook). If the reader is interested in the specifics of this methodology please refer to the LLT_Realized_Volatility notebook. The abriviated explanation is a sum of squared returns approach is used with resampling over a customaizable (set to 15 minutes in this notebook) timeframe using linear interpolation to populate missing values. When using the BS pricing model no dividend is assumed and interest rate is a parameter which has been currently set to 8%.

#### Noteable Tardis data issues
Coreum data only avalible on gate.io from 2023-01-22<br />
MIMO no data provided by tardis<br />
DEHUB data only avalible on gate.io from 2022-06-08<br />
Sologenic data only avalible on gate.io from 2022-06-08<br />
BTZ no data provided by tardis<br />
GTX no data provided by tardis<br />
UNB data only avalible on kucoin from 2022-08-16<br />

### Implementation: 

In [1]:
import pandas as pd
import numpy as np
from clickhouse_driver import Client
from scipy.stats import norm
import seaborn as sns
import matplotlib.pyplot as plt

# display package
from IPython.display import display, HTML, Markdown
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
class LLTVolatilityCapture():
    
    def __init__(self):
        
        # getting the clickhouse client
        try:
            # case running in an aws instance
            if socket.gethostbyaddr(socket.gethostname())[0].split('.')[1] == 'us-east-2':
                self.host_ip = '172.31.9.89'
        
            # case running on a local instance
            else:
                self.host_ip = '3.145.11.135'
        
        except:
            # case running on a local instance
            self.host_ip = '3.145.11.135'
        
        self.client = Client(host=self.host_ip,
                             port='9000',
                             database='test',
                             user='default',
                             compression=True,
                             connect_timeout=500_000,
                             send_receive_timeout=500_000,
                             sync_request_timeout=500_000,
                             settings={'use_numpy': True}
                             )
        
        # loading in the option terms from the clickhouse
        query = '''
            select *
            from
            test.llt_option_terms
            '''
        col = [
            'name',
            'option_size',
            'base_currency',
            'quote_currency',
            'strike',
            'start_date',
            'end_date'
            ]
        
        ot = self.client.execute(query)
        self.ot = pd.DataFrame(ot, columns=col)
        # cleaning the option terms data
        self.ot = self.ot[~self.ot.duplicated()]
        self.ot = self.ot[self.ot.start_date != '']
        self.ot = self.ot[self.ot.end_date != '']
        self.ot['option_size'] = self.ot['option_size'].astype('float')
        self.ot['strike'] = self.ot['strike'].astype('float')
        self.ot['start_date'] = pd.to_datetime(self.ot.start_date.str.strip(), format='%d/%m/%Y')
        self.ot['end_date'] = pd.to_datetime(self.ot.end_date.str.strip(), format='%d/%m/%Y')
        self.ot = self.ot[self.ot.end_date < pd.Timestamp.now()]
        self.ot.sort_values('end_date', inplace=True) # useful sort for merging
        
        # loading in the tardis instruments table
        query = f'''
        select *
        from
        test.tardis_instruments
        '''
        
        cols = [
            'instrument_id',
            'exchange',
            'symbol',
            'base',
            'quote',
            'instrument_type',
            'tardis_instrument_id',
            'tardis_exchange',
            'tardis_symbol'
        ]
        
        self.instruments = pd.DataFrame(self.client.execute(query), columns = cols)
        
        # loading in the option withdrawal table
        query = f'''
        select *
        from
        test.llt_option_withdrawal
        '''
        
        cols = [
            'name',
            'transfer_date',
            'withdrawal_amount'
        ]
        
        self.option_withdrawal = pd.DataFrame(self.client.execute(query), columns = cols)
        # cleaning the option withdrawl data
        self.option_withdrawal = self.option_withdrawal[~self.option_withdrawal.duplicated()]
        self.option_withdrawal = self.option_withdrawal[self.option_withdrawal.transfer_date != '']
        self.option_withdrawal['withdrawal_amount'] = self.option_withdrawal.withdrawal_amount.astype('float')
        self.option_withdrawal['transfer_date'] = pd.to_datetime(self.option_withdrawal.transfer_date.str.strip(), format='%d/%m/%Y')
        self.option_withdrawal.sort_values('transfer_date', inplace=True) # useful sort for merging
        
        # list of usd equivilents
        self.usd_stables = ['usd', 'usdt', 'usdc', 'busd']
        
        # setting seaborn plotting theme and figure size
        sns.set_theme(style="darkgrid")
        sns.set(rc={'figure.figsize':(20,4)})
        
        
    def generate_pnl(self, option_offset: pd.DateOffset = pd.DateOffset(days=14), time_offset: pd.DateOffset = pd.DateOffset(days=1)):
        """
        Attributes the PnL of the option to the various options using fuzzy matching logic between the option terms table and the option withdrawals table.
        The matching logic is as follows:
        Takes the option term and option withdrawal/deposit information ordered by option end date and withdrawal time respectively. For each client will look to attribute
        the withdrawal/deposit event as to a given option. This will then represent the PnL of the option. The attribution is done by looping through the clients options and 
        associated withdrawal/deposit events and attributing them to an option if the withdrawal event occurs after the option start_date but before the option end_date + 
        option_offset. Each withdrawal/deposit event can only be matched to a single option. To account for causes where for a given client multiple options occur at the same
        time the pnl is split uniformly between these options. For a given client two options are considered to occur at the same time if the start_dates and end_dates of the
        two options are within +/- the time_offset of eachother.
        Parameters
        -------
        option_offset - parameter used for fuzzy matching of withdrawal/deposit events and options, represents the time after an option expires but a withdrawal/deposit event
        will still be attribute to that given option.
        time_offset - parameter used for determination of when two options overlap. An option will overlap if the start_dates and end_dates of the two options are within
        +/- the time_offset.
        Returns
        -------
        None
        """
        # converts the withdrawal/deposit information into pnl information for each option by name and term of the option
        self.ot['PnL'] = 0
        # looping through the different option names, name represents a client
        for name in self.ot.name.unique():

            pnls = []
            option_withdrawal_filt = self.option_withdrawal[self.option_withdrawal.name == name].copy()
            ot_filt = self.ot.loc[self.ot.name == name]

            # loops through all options for the given client
            for otf_index, otf_row in ot_filt.iterrows():

                pnl = 0
                # loops through all withdrawal/deposit events for the given client
                for owf_index, owf_row in option_withdrawal_filt.iterrows():

                    # case that there are no withdrawl/deposit events move on to next client
                    if option_withdrawal_filt.shape[0] == 0:

                        break

                    # fuzzy macthing logic occurs here. A withdrawal/deposit event is attributed to the options PnL if it occurs during the options lifetime or before a pre determined
                    # time after the option expires, the option_offset parameter
                    if (owf_row['transfer_date'] >= otf_row['start_date']) and (owf_row['transfer_date'] <= otf_row['end_date'] + option_offset):

                        pnl += owf_row['withdrawal_amount']
                        # the withdrawal/deposit event is dropped to prevent douple counting of events
                        option_withdrawal_filt.drop(owf_index, inplace=True)

                pnls.append(pnl)

            self.ot.loc[(self.ot.name == name), 'PnL'] = pnls
            
        # for cases of more than one quote leg for an option attributes the pnl linearly between the different quote legs
        # looping through the different option names, name represents a client
        for name in lltvc.ot.name.unique():
    
            option_withdrawal_filt = lltvc.option_withdrawal[lltvc.option_withdrawal.name == name].copy()
            ot_filt = lltvc.ot.loc[(lltvc.ot.name == name)]
            start_date = None
            end_date = None

            prev_start_date = None
            prev_end_date = None
            
            # loops through all options for the given client
            for otf_index, otf_row in ot_filt.iterrows():

                start_date = otf_row['start_date']
                end_date = otf_row['end_date']

                # for the case we have already looped through an option
                if prev_start_date and prev_end_date:
                    
                    # checks to see if the current option start_date and end_date are within +- the time_offset if they are then move on to the next option if they are not
                    # then pnl attribution operations occur
                    start_trigger_filt = (start_date < prev_start_date - time_offset) or (start_date > prev_start_date + time_offset)
                    end_trigger_filt = (end_date < prev_end_date - time_offset) or (end_date > prev_end_date + time_offset)
                    if start_trigger_filt & end_trigger_filt:

                        # filter to only find those options which have a start_date and end_date which is within +- the time_offset. Logic being that these are the options which will
                        # share the pnl associated with the withdrawl/deposit events in the option period
                        start_filter = (ot_filt.start_date >= start_date - time_offset) & (ot_filt.start_date <= start_date + time_offset)
                        end_filter = (ot_filt.end_date >= end_date - time_offset) & (ot_filt.end_date <= end_date + time_offset)

                        # uniformly distributes the PnL between the relevant options
                        lltvc.ot.loc[(lltvc.ot.name == name)&(start_filter & end_filter), 'PnL'] = lltvc.ot.loc[(lltvc.ot.name == name)&(start_filter & end_filter), 'PnL'].replace(to_replace=0, method='ffill')
                        lltvc.ot.loc[(lltvc.ot.name == name)&(start_filter & end_filter), 'PnL'] = lltvc.ot.loc[(lltvc.ot.name == name)&(start_filter & end_filter), 'PnL']/ot_filt.loc[(start_filter & end_filter), 'PnL'].shape[0]

                    else:

                        pass

                # the case of evaluating the first option for a client always trigger the PnL attribution operations
                else:
                    
                    # filter to only find those options which have a start_date and end_date which is within +- the time_offset. Logic being that these are the options which will
                    # share the pnl associated with the withdrawl/deposit events in the option period
                    start_filter = (ot_filt.start_date >= start_date - time_offset) & (ot_filt.start_date <= start_date + time_offset)
                    end_filter = (ot_filt.end_date >= end_date - time_offset) & (ot_filt.end_date <= end_date + time_offset)
                    
                    # uniformly distributes the PnL between the relevant options
                    lltvc.ot.loc[(lltvc.ot.name == name)&(start_filter & end_filter), 'PnL'] = lltvc.ot.loc[(lltvc.ot.name == name)&(start_filter & end_filter), 'PnL'].replace(to_replace=0, method='ffill')
                    lltvc.ot.loc[(lltvc.ot.name == name)&(start_filter & end_filter), 'PnL'] = lltvc.ot.loc[(lltvc.ot.name == name)&(start_filter & end_filter), 'PnL']/ot_filt.loc[(start_filter & end_filter), 'PnL'].shape[0]

                # sets the previous start and ed dates to the current start and end dates
                prev_start_date = start_date
                prev_end_date = end_date
    
    
    def get_ohlc(self):
        """
        Generates relevant ohlc data used for conversion of options with non usd quote currencies into usd
        Returns
        -------
        ohlc - A Pandas dataframe of the relevant data avalible in tardis_ohlc_1m
        """
        
        # gets non usd quotes from various option agreements
        quotes = pd.Series(lltvc.ot.quote_currency.unique()).str.lower()
        quotes = quotes[~quotes.isin(lltvc.usd_stables)]
        tardis_instrument_ids = []
        for quote in quotes:
            tardis_instrument_ids.append(f'binance_{quote}_usdt_spot')
        
        # gets ohlc data for these markets to do usd conversion
        stingified_instrument_ids = []
        for instument in tardis_instrument_ids:
            stingified_instrument_ids.append("'" + instument + "'")

        query = f'''
        select *
        from
        test.tardis_ohlc_1m
        where
        instrument_id in ({', '.join(stingified_instrument_ids)})
        '''

        cols = [
            'instrument_id',
            'recieved_ts',
            'open',
            'high',
            'low',
            'close',
            'volume_base',
            'volume_quote'
        ]

        ohlc = pd.DataFrame(lltvc.client.execute(query), columns = cols)
        
        # pre processes the ohlc data
        ohlc['open'] = ohlc.open.astype(float)
        ohlc['high'] = ohlc.high.astype(float)
        ohlc['low'] = ohlc.low.astype(float)
        ohlc['close'] = ohlc.close.astype(float)
        ohlc['volume_base'] = ohlc.volume_base.astype(float)
        ohlc['volume_quote'] = ohlc.volume_quote.astype(float)
        ohlc['recieved_ts'] = pd.to_datetime(ohlc.recieved_ts.astype('int64'), unit = 'us')
        ohlc.sort_values('recieved_ts', inplace=True)
        
        return ohlc
    
    
    def get_trades(self, exchange_symbol: dict):
        """
        Pulls in the relevent trades from the tardis_publictrades table.
        
        Parameters
        -------
        exchange_symbol - Dictionary of the following format {exchange: {base: [quote_1, quote_2]}}.
        
        Returns
        -------
        trades - A Pandas dataframe of the trade data available in the tardis_publictrades table.
        """
        
        # generating the tardis instrument ids
        tardis_instrument_ids = []
        for exchange, market in exchange_symbol.items():
            for base, quotes in market.items():
                for quote in quotes:
                    tardis_instrument_ids.append(f'{exchange}_{base}_{quote}_spot')
        
        # getting the instrument id associated with these tardis instrument ids
        instruments_filt = self.instruments[self.instruments.tardis_instrument_id.isin(tardis_instrument_ids)]
        instrument_ids = instruments_filt.instrument_id.to_list()
        stingified_instrument_ids = []
        for instument in instrument_ids:
            stingified_instrument_ids.append("'" + instument + "'")

        # getting the relevent trade data
        query = f'''
        select *
        from
        test.tardis_publictrades_1m
        where
        instrument_id in ({', '.join(stingified_instrument_ids)})
        '''
        
        cols = [
            'instrument_id',
            'time_interval',
            'vwap_price',
            'total_amount'
        ]
        
        trades = pd.DataFrame(self.client.execute(query), columns = cols)
        
        # cleaning the trade data
        trades.rename(columns = {'vwap_price': 'price', 'total_amount': 'amount', 'time_interval': 'received_ts'}, inplace=True)
        trades['price'] = trades.price.astype(float)
        trades['amount'] = trades.amount.astype(float)
        trades['received_ts'] = pd.to_datetime(trades.received_ts.astype('int64'), unit = 's')
        trades.sort_values('received_ts', inplace=True)
        trades.set_index('received_ts', inplace = True)
        
        return trades
    
    
    def preprocess(self, exchange_symbol: dict, ohlc: pd.DataFrame()):
        """
        Pre processes the option terms data for the markets we are interested in. Specifically, selects only the option terms
        which are relevant for the base currencies we are interested in. Will then also check to see if within the relevant option
        terms there is an option with a quote currency which is USD and within the quotes argument there is a value which
        is a usd stable coin as defined by self.usd_stables. If this is the case, for each value of usd_stable in the quotes 
        parameter the option values with a quote currency which is USD are duplicated and the quote currency replaced with 
        the relevant usd_stable value. The logic being that in the case of this analysis I assume usd stable coins are equivilent
        to USD. As such if we have say a CCDUSD option we would be interested in the realized volatility not only on CCDUSD but
        also on say CCDUSDT. In the case that an options quote currency is a risky currency an exchange rate for this quote
        currency to usd is backed out using a VWAP of 1 minute ohlc data (using the open price) for this quote currency.
        
        Parameters
        -------
        exchange_symbol - Dictionary of the following format {exchange: {base: [quote_1, quote_2]}}.
        
        Returns
        -------
        ot_interested - The preprocessed option terms data.
        """
        
        # creating pairing of base currencies and quote currencies
        base_quote = {}
        for exchange, market in exchange_symbol.items():
            for base, quotes in market.items():
                if base in base_quote.keys():
                    base_quote[base].add(set(quotes))
                else:
                    base_quote[base] = set(quotes)
        
        ot_interested = []
        # cycling through all avalible base and quote pairing
        for base, quotes in base_quote.items():
                    
            quotes = list(quotes)
            # selecting option term data that is relevent for the market we are examining
            if pd.Series(quotes).isin(self.usd_stables).any(): 
                ot_filt = self.ot.loc[(self.ot.base_currency.str.lower() == base) & (self.ot.quote_currency.str.lower().isin(quotes+['usd']))].copy()
                
            else:  
                ot_filt = self.ot.loc[(self.ot.base_currency.str.lower() == base) & (self.ot.quote_currency.str.lower().isin(quotes))].copy()
            
            # checks for case of USD option and USD stablecoin markets
            relevant_stables = []
            for symbol in self.usd_stables:
                if symbol in quotes:
                    relevant_stables.append(symbol)

            # case of USD option and usd stablecoin markets adds rows to ot_filt for these markets
            if ((ot_filt.quote_currency == 'USD').count() > 0) and (len(relevant_stables) > 0):

                # uniformly distributes the PnL and option size between the relevant options
                num = len(relevant_stables)
                ot_filt.loc[(ot_filt.quote_currency == 'USD'),'PnL'] = ot_filt.loc[(ot_filt.quote_currency == 'USD'),'PnL']/num
                ot_filt.loc[(ot_filt.quote_currency == 'USD'),'option_size'] = ot_filt.loc[(ot_filt.quote_currency == 'USD'),'option_size']/num
                for stable in relevant_stables:

                    dup = ot_filt[ot_filt.quote_currency == 'USD'].copy()
                    dup['quote_currency'] = stable.upper()
                    ot_interested.append(dup)

            # getting relevant non usd quote markets
            relevant_riskies = []
            for quote in quotes:
                if quote not in self.usd_stables:
                    relevant_riskies.append(quote.lower())
                    
            if len(relevant_riskies) > 0:
                risky_dup = ot_filt.loc[ot_filt.quote_currency.str.lower().isin(relevant_riskies)].copy()
                ot_interested.append(risky_dup)
            
        ot_interested = pd.concat(ot_interested)
        ot_interested.sort_values(['start_date', 'quote_currency'], ascending=True, inplace=True)
        ot_interested.reset_index(inplace=True, drop=True)
        
        # computing quote/usd exchange rate for non usd equivalent quote currencies
        ot_interested['quote_usd_rate'] = 1
        for oti_index, oti_row in ot_interested.iterrows():
            
            if oti_row['quote_currency'].lower() not in self.usd_stables:
                
                instrument_id = f'binance_{oti_row["quote_currency"].lower()}_usdt_spot'
                filt = (ohlc.instrument_id == instrument_id) & (ohlc.recieved_ts>=oti_row['start_date']) & (ohlc.recieved_ts<=oti_row['end_date'])
                ohlc_filt = ohlc.loc[filt]
                exchange_rate = (ohlc_filt.open*ohlc.volume_base).sum()/ohlc_filt.volume_base.sum()
                ot_interested.loc[oti_index, 'quote_usd_rate'] = exchange_rate
            
        return ot_interested
    
    
    def generate_volatility(self, trades: pd.DataFrame(), ot_interested: pd.DataFrame(), minutes: int):
        """
        Generates the realized volatility statistics for the different options.
        
        Parameters
        -------
        trades - Pandas dataframe of all relevent tardis trades.
        ot_interested - The preprosessed options terms dataframe.
        minutes - The minutes window over which resampling occurs for the volatility metric.
        
        Returns
        -------
        ot_interested - The preprosessed options terms dataframe with realized volatiltiy.
        """
        
        # holds the realized vol numbers
        volatility = []
        # looping through the relevant options data
        for start_date, end_date, base, quote in zip(ot_interested['start_date'], ot_interested['end_date'], ot_interested['base_currency'], ot_interested['quote_currency']):
            
            # filtering for the relevant market for this option
            symbol = (base + '_' + quote).lower()
            df_filt = trades[trades.instrument_id.str.contains(symbol)&(trades.index>=start_date)&(trades.index<=end_date)].copy()
            # deletes duplicates based on recieved ts taking the result with the lowest id and then resamples using a time window given by minutes
            # linear interpolation then used to populate the sparce dataframe of prices
            df_filt = df_filt[~df_filt.index.duplicated()][['price']].resample(f'{minutes}T').first().ffill(limit=1).interpolate('linear')
            # returns computed
            df_filt['return'] = (df_filt.price.diff()/df_filt.price.shift(1))
            # volatility is computed and annualised
            volatility.append(np.sqrt((df_filt['return']**2).sum()/(df_filt[~df_filt['return'].isna()].shape[0] - 1))*np.sqrt(365*24*60/minutes))
            
        ot_interested['realized_volatility'] = volatility
        
        return ot_interested[['base_currency', 'quote_currency', 'start_date', 'end_date', 'option_size', 'strike', 'quote_usd_rate', 'PnL', 'realized_volatility']]
    
    
    def bs_call(self, ot_interested: pd.DataFrame, interest_rate: float):
        """
        Computes the black scholes call price for the different option using the realized volatiltiy over the lifetime of the option. Several assumptions made:
        1) Spot price at inception equal to strike price.
        2) No dividend rate.
        
        Parameters
        -------
        ot_interested - The preprosessed options terms dataframe.
        interest_rate - The annualized continuous interest rate.
        
        Returns
        -------
        ot_interested - The preprosessed options terms dataframe with the call value.
        """
        time = (ot_interested.end_date - ot_interested.start_date).dt.days/365.25
        d1 = (interest_rate + ot_interested.realized_volatility**2/2)*time/(ot_interested.realized_volatility*time**(1/2))
        d2 = d1 - ot_interested.realized_volatility*time**(1/2)
        ot_interested['call_value'] = ot_interested.strike*pd.Series(norm.cdf(d1)) - ot_interested.strike*np.exp(-interest_rate*time)*pd.Series(norm.cdf(d2))
        ot_interested['call_value'] = ot_interested.call_value*ot_interested.option_size*ot_interested.quote_usd_rate
        
        return ot_interested
    
    
    def volatility_capture(self, ot_interested: pd.DataFrame):
        """
        Generates our metric of volatility capture for the relevant options.
        
        Parameters
        -------
        ot_interested - The preprosessed options terms dataframe.
        
        Returns
        -------
        ot_interested - The preprosessed options terms dataframe with volatility capture.
        ot_agg - The preprosessed options aggregated by option agreement.
        """
        
        # generating volaitltiy capture metric
        ot_interested['volatility_capture'] = ot_interested.PnL/ot_interested.call_value
        
        # aggregating by option agreement
        cols = ['option_size', 'PnL', 'call_value']
        ot_agg = ot_interested.groupby(['base_currency', pd.Grouper(key='start_date', freq='w'), pd.Grouper(key='end_date', freq='w')])[cols].sum()
        ot_agg['volatility_capture'] = ot_agg.PnL/ot_agg.call_value
        
        return ot_interested, ot_agg
    
    
    def display_volatility_capture(self, exchange_symbol: dict, minutes: int, interest_rate: float):
        """
        Convenient wrapper to call methods in correct order to generate the volatility capture metric.

        Parameters
        -------
        exchange_symbol - Dictionary of the following format {exchange: {base: [quote_1, quote_2]}}.
        minutes - The minutes window over which resampling occurs for the volatility metric.
        interest_rate - The annualized continuous interest rate.

        Returns
        -------
        ot_interested - The preprosessed options terms dataframe with volatility capture.
        ot_agg - The preprosessed options aggregated by option agreement with volatility capture.
        """
        self.generate_pnl()
        ohlc = self.get_ohlc()
        trades = self.get_trades(exchange_symbol)
        ot_interested = self.preprocess(exchange_symbol, ohlc)
        ot_interested = self.generate_volatility(trades, ot_interested, minutes)
        ot_interested = self.bs_call(ot_interested, interest_rate)
        ot_interested, ot_agg = self.volatility_capture(ot_interested)
        ot_interested.sort_values(['base_currency', 'start_date', 'quote_currency'], inplace=True)
        ot_interested.set_index(['base_currency', 'quote_currency', 'start_date', 'end_date'], inplace=True)

        display(Markdown(f"### Volatility capture by individual option:"))
        display(ot_interested)
        display(Markdown(f"### Volatility capture by option agreement:"))
        display(ot_agg)

        return ot_interested, ot_agg

Parameters:

In [3]:
exchange_symbol = {
    'okex': {
        'velo': ['usdt']
    },
    'binance': {
        'ant': ['usdt', 'btc']
    },
    'bitfinex': {
        'ccd': ['usd', 'usdt', 'btc']
    }
}
interest_rate = 0.08
minutes = 15

In [4]:
lltvc = LLTVolatilityCapture()
ot_interested, ot_agg = lltvc.display_volatility_capture(exchange_symbol, minutes, interest_rate)

### Volatility capture by individual option:

option_size        strike  \
base_currency quote_currency start_date end_date                                
ANT           BTC            2022-01-15 2022-07-14      25000.0  1.976650e-04   
              USDT           2022-01-15 2022-07-14      25000.0  8.402857e+00   
              BTC            2022-07-15 2023-01-15      25000.0  7.980000e-05   
              USDT           2022-07-15 2023-01-15      25000.0  1.648571e+00   
CCD           BTC            2022-02-10 2022-08-10   10000000.0  1.910000e-06   
              USD            2022-02-11 2022-08-11    5000000.0  8.783060e-02   
              USDT           2022-02-11 2022-08-11    5000000.0  8.783060e-02   
              BTC            2022-08-10 2023-02-10   10000000.0  7.810000e-07   
              USD            2022-08-11 2023-02-11    5000000.0  1.804857e-02   
              USDT           2022-08-11 2023-02-11    5000000.0  1.804857e-02   
VELO          USDT           2022-02-14 2022-03-14    2273283.8  7.485207e-02   
                             2022-03-14 2022-04-14    3914256.2  4.191481e-02   
                             2022-04-14 2022-05-14    4068307.5  4.032766e-02   
                             2022-05-14 2022-06-14    4900826.5  3.182830e-02   
                             2022-06-14 2022-07-14    4693084.0  3.328834e-02   
                             2022-07-14 2022-08-14    6695017.5  2.248314e-02   
                             2022-08-14 2022-09-14    7302311.0  2.108168e-02   
                             2022-09-14 2022-10-14   18686318.0  8.010406e-03   
                             2022-10-14 2022-11-14   29203938.0  5.017131e-03   
                             2022-11-14 2022-12-14   79850410.0  1.940128e-03   
                             2022-12-14 2023-01-14  141503570.0  1.126191e-03   
                             2023-01-14 2023-02-14   84839100.0  1.919044e-03   
                             2023-02-14 2023-03-14   67178440.0  2.412158e-03   

                                                    quote_usd_rate  \
base_currency quote_currency start_date end_date                     
ANT           BTC            2022-01-15 2022-07-14    30936.124251   
              USDT           2022-01-15 2022-07-14        1.000000   
              BTC            2022-07-15 2023-01-15    19414.256574   
              USDT           2022-07-15 2023-01-15        1.000000   
CCD           BTC            2022-02-10 2022-08-10    28467.814279   
              USD            2022-02-11 2022-08-11        1.000000   
              USDT           2022-02-11 2022-08-11        1.000000   
              BTC            2022-08-10 2023-02-10    19618.286384   
              USD            2022-08-11 2023-02-11        1.000000   
              USDT           2022-08-11 2023-02-11        1.000000   
VELO          USDT           2022-02-14 2022-03-14        1.000000   
                             2022-03-14 2022-04-14        1.000000   
                             2022-04-14 2022-05-14        1.000000   
                             2022-05-14 2022-06-14        1.000000   
                             2022-06-14 2022-07-14        1.000000   
                             2022-07-14 2022-08-14        1.000000   
                             2022-08-14 2022-09-14        1.000000   
                             2022-09-14 2022-10-14        1.000000   
                             2022-10-14 2022-11-14        1.000000   
                             2022-11-14 2022-12-14        1.000000   
                             2022-12-14 2023-01-14        1.000000   
                             2023-01-14 2023-02-14        1.000000   
                             2023-02-14 2023-03-14        1.000000   

                                                           PnL  \
base_currency quote_currency start_date end_date                 
ANT           BTC            2022-01-15 2022-07-14   52950.975   
              USDT           2022-01-15 2022-07-14   52950.975   
           

### Volatility capture by option agreement:

option_size         PnL     call_value  \
base_currency start_date end_date                                             
ANT           2022-01-16 2022-07-17      50000.0  105901.950  151931.897483   
              2022-07-17 2023-01-15      50000.0   20000.000   28514.840638   
CCD           2022-02-13 2022-08-14   20000000.0   12906.000  467552.382183   
              2022-08-14 2023-02-12   20000000.0   42727.000   74536.521301   
VELO          2022-02-20 2022-03-20    2273283.8   10939.551   29219.068350   
              2022-03-20 2022-04-17    3914256.2   67064.000   25775.156861   
              2022-04-17 2022-05-15    4068307.5   38980.043   38498.040390   
              2022-05-15 2022-06-19    4900826.5   19171.000   43640.037320   
              2022-06-19 2022-07-17    4693084.0    5559.000   26880.505308   
              2022-07-17 2022-08-14    6695017.5    9015.705   20326.220671   
              2022-08-14 2022-09-18    7302311.0   14908.000   22857.594747   
              2022-09-18 2022-10-16   18686318.0   23943.145   29684.080815   
              2022-10-16 2022-11-20   29203938.0   22832.000   33985.738061   
              2022-11-20 2022-12-18   79850410.0   16623.330   34235.449540   
              2022-12-18 2023-01-15  141503570.0   99310.334   47014.978192   
              2023-01-15 2023-02-19   84839100.0  536333.000   35393.550915   
              2023-02-19 2023-03-19   67178440.0  100000.000   60657.065959   

                                     volatility_capture  
base_currency start_date end_date                        
ANT           2022-01-16 2022-07-17            0.697036  
              2022-07-17 2023-01-15            0.701389  
CCD           2022-02-13 2022-08-14            0.027603  
              2022-08-14 2023-02-12            0.573236  
VELO          2022-02-20 2022-03-20            0.374398  
              2022-03-20 2022-04-17            2.601885  
              2022-04-17 2022-05-15            1.012520  
              2022-05-15 2022-06-19            0.439298  
              2022-06-19 2022-07-17            0.206804  
              2022-07-17 2022-08-14            0.443550  
              2022-08-14 2022-09-18            0.652212  
              2022-09-18 2022-10-16            0.806599  
              2022-10-16 2022-11-20            0.671811  
              2022-11-20 2022-12-18            0.485559  
              2022-12-18 2023-01-15            2.112313  
              2023-01-15 2023-02-19           15.153410  
              2023-02-19 2023-03-19            1.648613